## Detailed article explaination

The detailed code explanation for this article is available at the following link:

https://www.daniweb.com/programming/computer-science/tutorials/542680/evaluating-gpt-4o-november-model-for-text-classification-and-summarization

For my other articles for Daniweb.com, please see this link:

https://www.daniweb.com/members/1235222/usmanmalik57

## Importing and Installing Required Libraries

In [ ]:
!pip install openai
!pip install rouge-score
!pip install --upgrade openpyxl
!pip install pandas openpyxl

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from collections import Counter
from sklearn.metrics import hamming_loss, accuracy_score
from rouge_score import rouge_scorer
from openai import OpenAI

from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [ ]:
client = OpenAI(api_key = OPENAI_API_KEY)

## Comparison for Text Summarization

In [ ]:

# Kaggle dataset download link
# https://github.com/reddzzz/DataScience_FP/blob/main/dataset.xlsx


dataset = pd.read_excel(r"/content/dataset.xlsx")
print(dataset.shape)
dataset.head()

(1000, 10)


,Unnamed: 0,id,human_summary,publication,author,date,year,month,theme,content
0,0,17283,In successfully seeking a temporary halt in th...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,politics,WASHINGTON — Congressional Republicans have...
1,0,17284,Officers put her in worse danger some months l...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,crime,"After the bullet shells get counted, the blood..."
2,0,17285,The film striking appearance had been created ...,New York Times,Margalit Fox,2017-01-06,2017.0,1.0,entertainment,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,0,17286,The year was only days old when the news came ...,New York Times,William McDonald,2017-04-10,2017.0,4.0,entertainment,"Death may be the great equalizer, but it isn’t..."
4,0,17287,If North Korea conducts a test in coming month...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,politics,"SEOUL, South Korea — North Korea’s leader, ..."


In [ ]:
# Function to calculate ROUGE scores
def calculate_rouge(reference, candidate):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, candidate)
    return {key: value.fmeasure for key, value in scores.items()}

In [ ]:
def summarize_articles_with_model(model_id):


    results = []
    for i, (_, row) in enumerate(dataset[:20].iterrows(), start=1):
        article = row['content']
        human_summary = row['human_summary']

        print(f"Summarizing article {i}.")

        prompt = f"Summarize the following article in 1150 characters. The summary should look like human created:\n\n{article}\n\nSummary:"

        response = client.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=1150,
            temperature=0
        )
        generated_summary = response.choices[0].message.content
        rouge_scores = calculate_rouge(human_summary, generated_summary)

        results.append({
            'article_id': row.id,
            'generated_summary': generated_summary,
            'rouge1': rouge_scores['rouge1'],
            'rouge2': rouge_scores['rouge2'],
            'rougeL': rouge_scores['rougeL']
        })

    return results


### Summarization Results with GPT-4o November Update

In [ ]:
results = summarize_articles_with_model("gpt-4o-2024-11-20")
results_df = pd.DataFrame(results)
mean_values = results_df[["rouge1", "rouge2", "rougeL"]].mean()
print(mean_values)

Summarizing article 1.
Summarizing article 2.
Summarizing article 3.
Summarizing article 4.
Summarizing article 5.
Summarizing article 6.
Summarizing article 7.
Summarizing article 8.
Summarizing article 9.
Summarizing article 10.
Summarizing article 11.
Summarizing article 12.
Summarizing article 13.
Summarizing article 14.
Summarizing article 15.
Summarizing article 16.
Summarizing article 17.
Summarizing article 18.
Summarizing article 19.
Summarizing article 20.
rouge1    0.341660
rouge2    0.069531
rougeL    0.150887
dtype: float64


### Summarization Results with GPT-4o August Update

In [ ]:
results = summarize_articles_with_model("gpt-4o-2024-08-06")
results_df = pd.DataFrame(results)
mean_values = results_df[["rouge1", "rouge2", "rougeL"]].mean()
print(mean_values)

Summarizing article 1.
Summarizing article 2.
Summarizing article 3.
Summarizing article 4.
Summarizing article 5.
Summarizing article 6.
Summarizing article 7.
Summarizing article 8.
Summarizing article 9.
Summarizing article 10.
Summarizing article 11.
Summarizing article 12.
Summarizing article 13.
Summarizing article 14.
Summarizing article 15.
Summarizing article 16.
Summarizing article 17.
Summarizing article 18.
Summarizing article 19.
Summarizing article 20.
rouge1    0.363209
rouge2    0.086697
rougeL    0.170481
dtype: float64


## Comparison for Multi-Class Zero-Shot Text Classification

In [ ]:
## Dataset download link
## https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment?select=Tweets.csv

dataset = pd.read_csv(r"/content/Tweets.csv")
print(dataset.shape)
dataset.head()

(14640, 15)


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [ ]:
# Remove rows where 'airline_sentiment' or 'text' are NaN
dataset = dataset.dropna(subset=['airline_sentiment', 'text'])

# Remove rows where 'airline_sentiment' or 'text' are empty strings
dataset = dataset[(dataset['airline_sentiment'].str.strip() != '') & (dataset['text'].str.strip() != '')]

# Filter the DataFrame for each sentiment
neutral_df = dataset[dataset['airline_sentiment'] == 'neutral']
positive_df = dataset[dataset['airline_sentiment'] == 'positive']
negative_df = dataset[dataset['airline_sentiment'] == 'negative']

# Randomly sample records from each sentiment
neutral_sample = neutral_df.sample(n=34)
positive_sample = positive_df.sample(n=33)
negative_sample = negative_df.sample(n=33)

# Concatenate the samples into one DataFrame
dataset = pd.concat([neutral_sample, positive_sample, negative_sample])

# Reset index if needed
dataset.reset_index(drop=True, inplace=True)

# print value counts
print(dataset["airline_sentiment"].value_counts())


airline_sentiment
neutral     34
positive    33
negative    33
Name: count, dtype: int64


In [ ]:

def find_sentiment(client, model):

    tweets_list = dataset["text"].tolist()


    all_sentiments = []

    i = 0
    exceptions = 0
    while i < len(tweets_list):

        try:
            tweet = tweets_list[i]
            content = """What is the sentiment expressed in the following tweet about an airline?
            Select sentiment value from positive, negative, or neutral. Return only the sentiment value in small letters.
            tweet: {}""".format(tweet)

            sentiment_value = client.chat.completions.create(
                                  model= model,
                                  temperature = 0,
                                  max_tokens = 10,
                                  messages=[
                                        {"role": "user", "content": content}
                                    ]
                                ).choices[0].message.content

            all_sentiments.append(sentiment_value)
            i = i + 1
            print(i, sentiment_value)

        except Exception as e:
            print("===================")
            print("Exception occurred:", e)
            exceptions += 1

    print("Total exception count:", exceptions)
    accuracy = accuracy_score(all_sentiments, dataset["airline_sentiment"])
    print("Accuracy:", accuracy)


### Multi-Class Text Classification Results with GPT-4o November Update

In [ ]:
model = "gpt-4o-2024-11-20"
find_sentiment(client, model)

1 positive
2 positive
3 neutral
4 neutral
5 neutral
6 neutral
7 neutral
8 negative
9 neutral
10 neutral
11 negative
12 positive
13 negative
14 neutral
15 negative
16 neutral
17 negative
18 neutral
19 negative
20 neutral
21 neutral
22 negative
23 negative
24 negative
25 neutral
26 neutral
27 neutral
28 negative
29 neutral
30 negative
31 neutral
32 neutral
33 neutral
34 neutral
35 positive
36 positive
37 positive
38 positive
39 negative
40 positive
41 positive
42 positive
43 positive
44 negative
45 positive
46 positive
47 neutral
48 positive
49 positive
50 positive
51 positive
52 positive
53 positive
54 positive
55 positive
56 positive
57 positive
58 positive
59 positive
60 positive
61 neutral
62 positive
63 neutral
64 positive
65 positive
66 positive
67 positive
68 negative
69 negative
70 negative
71 negative
72 negative
73 negative
74 negative
75 negative
76 negative
77 negative
78 negative
79 negative
80 neutral
81 negative
82 negative
83 negative
84 negative
85 negative
86 negative
8

### Multi-Class Text Classification Results with GPT-4o August Update

In [ ]:
model = "gpt-4o-2024-08-06"
find_sentiment(client, model)

1 positive
2 positive
3 neutral
4 neutral
5 neutral
6 neutral
7 neutral
8 negative
9 neutral
10 neutral
11 negative
12 neutral
13 negative
14 neutral
15 negative
16 neutral
17 negative
18 neutral
19 negative
20 neutral
21 neutral
22 negative
23 neutral
24 negative
25 neutral
26 neutral
27 neutral
28 negative
29 negative
30 negative
31 neutral
32 neutral
33 neutral
34 neutral
35 negative
36 positive
37 positive
38 positive
39 negative
40 positive
41 positive
42 positive
43 neutral
44 negative
45 positive
46 positive
47 neutral
48 positive
49 positive
50 positive
51 neutral
52 neutral
53 positive
54 positive
55 positive
56 positive
57 positive
58 positive
59 positive
60 neutral
61 neutral
62 positive
63 neutral
64 positive
65 positive
66 positive
67 positive
68 negative
69 negative
70 negative
71 negative
72 negative
73 negative
74 negative
75 negative
76 negative
77 negative
78 negative
79 negative
80 neutral
81 negative
82 negative
83 negative
84 negative
85 negative
86 negative
87 neg

## Comparison for Multi-label Zero-Shot Text Classification

In [ ]:
## dataset download link
## https://www.kaggle.com/datasets/shivanandmn/multilabel-classification-dataset?select=train.csv

dataset = pd.read_csv(r"/content/train.csv")
print(f"Dataset Shape: {dataset.shape}")
dataset.head()

Dataset Shape: (20972, 9)


,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


In [ ]:
subjects = ["Computer Science", "Physics", "Mathematics", "Statistics", "Quantitative Biology", "Quantitative Finance"]
filtered_dataset = dataset[(dataset[subjects] == 1).sum(axis=1) >= 2]
print(f"Filtered Dataset Shape: {filtered_dataset.shape}")
filtered_dataset.head()

Filtered Dataset Shape: (5044, 9)


,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0
21,22,Many-Body Localization: Stability and Instability,Rare regions with weak disorder (Griffiths r...,0,1,1,0,0,0
28,29,Minimax Estimation of the $L_1$ Distance,We consider the problem of estimating the $L...,0,0,1,1,0,0
29,30,Density large deviations for multidimensional ...,We investigate the density large deviation f...,0,1,1,0,0,0
30,31,mixup: Beyond Empirical Risk Minimization,"Large deep neural networks are powerful, but...",1,0,0,1,0,0


In [ ]:

def find_research_category(client, model, dataset):

    outputs = []
    i = 0

    for _, row in dataset.iterrows():
        title = row['TITLE']
        abstract = row['ABSTRACT']

        content = """You are an expert in various scientific domains.
                     Given the following research paper title and abstract, classify the research paper into at least two or more of the following categories:
                    - Computer Science
                    - Physics
                    - Mathematics
                    - Statistics
                    - Quantitative Biology
                    - Quantitative Finance

                    Return only a comma-separated list of the categories (e.g., [Computer Science,Physics] or [Computer Science,Physics,Mathematics]).
                    Use the exact case sensitivity and spelling of the categories provided above.

                    text: Title: {}\nAbstract: {}""".format(title, abstract)


        research_category = client.chat.completions.create(
                                model= model,
                                temperature = 0,
                                max_tokens = 100,
                                messages=[
                                      {"role": "user", "content": content}
                                  ]
                              ).choices[0].message.content


        outputs.append(research_category)
        print(i + 1, research_category)
        i += 1

    return outputs


In [ ]:

def parse_outputs_to_dataframe(outputs):

    subjects = ["Computer Science", "Physics", "Mathematics", "Statistics", "Quantitative Biology", "Quantitative Finance"]
    # Remove square brackets and split the subjects for each entry in outputs
    parsed_data = [item.strip('[]').split(',') for item in outputs]

    # Create an empty DataFrame with columns for each subject, initializing with 0s
    df = pd.DataFrame(0, index=range(len(parsed_data)), columns=subjects)

    # Populate the DataFrame with 1s based on the presence of each subject in each row
    for i, subjects_list in enumerate(parsed_data):
        for subject in subjects_list:
            if subject in subjects:
                df.loc[i, subject] = 1

    return df

In [ ]:
sampled_df = filtered_dataset.sample(n=100, random_state=42)

### Multi-Class Text Classification Results with GPT-4o November Update

In [ ]:

model = "gpt-4o-2024-11-20"
outputs = find_research_category(client,
                                 model,
                                 sampled_df)

predictions = parse_outputs_to_dataframe(outputs)
targets = sampled_df[subjects]

# Calculate Hamming Loss
hamming = hamming_loss(targets, predictions)
print(f"Hamming Loss: {hamming}")

# Calculate Subset Accuracy (Exact Match Ratio)
subset_accuracy = accuracy_score(targets, predictions)
print(f"Subset Accuracy: {subset_accuracy}")

1 [Physics,Mathematics]
2 [Statistics,Quantitative Biology]
3 [Computer Science,Statistics]
4 [Mathematics,Statistics]
5 [Computer Science,Statistics]
6 [Mathematics,Statistics]
7 [Computer Science,Mathematics]
8 [Statistics,Quantitative Biology]
9 [Computer Science,Mathematics,Statistics]
10 [Computer Science,Mathematics,Statistics]
11 [Computer Science,Statistics]
12 [Computer Science,Statistics,Mathematics]
13 [Computer Science,Physics,Statistics]
14 [Computer Science,Statistics]
15 [Computer Science,Statistics]
16 [Statistics,Mathematics]
17 [Mathematics,Statistics]
18 [Statistics,Quantitative Biology]
19 [Computer Science,Statistics,Mathematics]
20 [Mathematics,Statistics]
21 [Statistics,Quantitative Biology]
22 [Mathematics]
23 [Mathematics,Statistics]
24 [Computer Science,Statistics]
25 [Mathematics,Statistics]
26 [Computer Science,Statistics]
27 [Computer Science,Mathematics]
28 [Computer Science,Quantitative Biology]
29 [Physics,Mathematics]
30 [Computer Science,Mathematics]
3

### Multi-Class Text Classification Results with GPT-4o August Update

In [ ]:
model = "gpt-4o-2024-08-06"
outputs = find_research_category(client,
                                 model,
                                 sampled_df)

predictions = parse_outputs_to_dataframe(outputs)
targets = sampled_df[subjects]

# Calculate Hamming Loss
hamming = hamming_loss(targets, predictions)
print(f"Hamming Loss: {hamming}")

# Calculate Subset Accuracy (Exact Match Ratio)
subset_accuracy = accuracy_score(targets, predictions)
print(f"Subset Accuracy: {subset_accuracy}")


1 [Physics,Mathematics]
2 [Statistics]
3 [Computer Science,Statistics]
4 [Statistics,Mathematics]
5 [Computer Science,Statistics]
6 [Statistics,Mathematics]
7 Mathematics,Computer Science,Statistics
8 [Statistics,Quantitative Finance]
9 [Computer Science, Mathematics, Statistics]
10 [Computer Science, Mathematics, Statistics]
11 [Computer Science,Statistics]
12 [Computer Science,Statistics]
13 [Computer Science,Physics]
14 [Computer Science,Statistics]
15 [Computer Science,Statistics]
16 [Statistics,Quantitative Biology]
17 [Mathematics,Statistics]
18 [Statistics, Quantitative Biology]
19 [Computer Science,Statistics]
20 Mathematics,Statistics
21 [Statistics,Quantitative Biology]
22 [Mathematics]
23 [Statistics,Mathematics]
24 [Computer Science]
25 [Mathematics,Statistics]
26 [Computer Science,Statistics]
27 [Computer Science,Mathematics]
28 [Computer Science, Quantitative Biology]
29 [Physics,Mathematics]
30 [Computer Science,Statistics]
31 [Computer Science,Mathematics,Statistics]
32